This notebook implements HDBSCAN (Hierarchical Density-Based Spatial Clustering of Applications with Noise) on time series data after extracting a comprehensive set of features in the frequency domain. This method focuses on finding clusters of varying densities and shapes within the feature space, automatically identifying outliers as noise (labeled -1).Unlike K-Means, HDBSCAN doesn't require predefining the number of clusters and is more robust to noise, making it suitable for exploratory analysis of complex data distributions. The clustering is based on the Euclidean distance applied to the features.The complete workflow covered in the notebook is as follows:
1. Data Loading and Preprocessing: Loads the raw multivariate data and applies Z-normalization to each individual time series.
2. Feature Extraction (FFT): Computes the amplitude spectrum using the Fast Fourier Transform (FFT) and extracts the N most dominant low-frequency coefficients (amplitude spectrum, excluding the DC component) to create the feature set.
3. Feature Scaling: Applies Standard Scaling (Z-score normalization) to the extracted feature matrix
4. Dimensionality Reduction (PCA): Applies Principal Component Analysis (PCA) to the scaled feature matrix to decorrelate features and reduce dimensionality.
5. Clustering Execution: Runs the HDBSCAN algorithm on the reduced feature matrix, using its core density-based hyperparameters (min_cluster_size and min_samples).
6. Evaluation: Computes external metrics (ARI, NMI, Purity) on the non-noise points, and an internal metric based on the Euclidean distance (Euclidean Silhouette Score).
7. Visualization: Uses UMAP for dimensionality reduction on the feature matrix and plots the 2D embedding colored by cluster assignments and noise points.

1. Import Libraries 

In [ ]:
import sys; sys.path.append("..")
import numpy as np
from src.io import load_dataset
from src.preprocess import z_norm_per_series
from src.eval import external_scores, confusion_with_hungarian, silhouette_by_metric
from src.viz import embed_tsne_from_dist, scatter_2d, plot_confusion_matrix
from src.utils_io import save_clustering_results
from src.fft import compute_fft_spectrum, extract_fourier_features
from src.time_features import compute_time_features, scale_features, concatenate_features
from src.distances import distance_matrix
from src.reducers import apply_umap
from src.hdbscan import run_hdbscan
import time
from itertools import product
import pandas as pd

2. Define Hyperparameters

In [2]:
SAMPLING_RATE = 51.2  
N_FOURIER_COEFFS = 16
N_COMPONENTS = 2
N_NEIGHBORS = 15
MIN_DIST = 0.1  
METRIC_UMAP = 'euclidean'
RANDOM_STATE = 42
MIN_CLUSTER_SIZE = 15
MIN_SAMPLES = 5
METRIC_HDBSCAN="euclidean"

3. Load Data and preprocessign

In [3]:
# Load dataset
X_train, X_test, y_train, y_test = load_dataset("../data/raw", test_size=0.2, random_state=42)

# Preprocessing: z-normalization (per series)
Xn_train = z_norm_per_series(X_train)
Xn_test  = z_norm_per_series(X_test)

3. Extract Fourier Features

In [ ]:
freqs, amp = compute_fft_spectrum(
    X=Xn_train, 
    sampling_rate=SAMPLING_RATE, 
    center=True
)
X_freq = extract_fourier_features(amp, n_coeffs=N_FOURIER_COEFFS)

4. Feature Engineering

In [ ]:
X_time = compute_time_features(Xn_train)

5. Feature Concatenationand scaling

In [ ]:
X_feat = concatenate_features(X_time, X_freq)
X_feat_scaled = scale_features(X_feat)

6. Apply UMAP for dimensionality reduction

In [ ]:
X_umap = apply_umap(
    X=X_feat_scaled, 
    n_components=N_COMPONENTS, 
    n_neighbors=N_NEIGHBORS, 
    min_dist=MIN_DIST, 
    metric=METRIC_UMAP, 
    random_state=RANDOM_STATE
)

7. Apply HDBSCAN

In [ ]:
y_pred, clusterer = run_hdbscan(
    X=X_umap, 
    min_cluster_size=MIN_CLUSTER_SIZE, 
    min_samples=MIN_SAMPLES, 
    metric=METRIC_HDBSCAN
)

6. Evaluation

In [ ]:
mask = y_pred != -1
scores = external_scores(y_train[mask], y_pred[mask])
print("External metrics (train):", scores)

sil = silhouette_by_metric(X_umap[mask], y_pred[mask], metric=METRIC_HDBSCAN)
print(f"Silhouette (UMAP, sans bruit) : {sil:.3f}")

#Mapping & confusion
y_pred_mapped, C, mapping = confusion_with_hungarian(y_train[mask], y_pred[mask])
mapping_clean = {int(k): int(v) for k, v in mapping.items()}

#Visualization (SBD)
D = distance_matrix(X_umap)                 
X2 = embed_tsne_from_dist(D, perplexity=30, random_state=42)
fig = scatter_2d(X2, y_pred, "k-Shape — t-SNE on SBD distances", y_true=y_train[mask], show=True)

#Confusion matrix
true_labels = np.sort(np.unique(y_train[mask]))
pred_labels = np.sort(np.unique(y_pred[mask]))
fig = plot_confusion_matrix(
    C, true_labels, pred_labels,
    label_to_activity={1:"Walk",2:"Stairs Up",3:"Stairs Down",4:"Sit",5:"Stand",6:"Lay"},
    normalize=True,
    title="k-Shape Confusion Matrix (Hungarian Aligned)",
    show=True
)

7. Grid Search 

In [ ]:
RESULTS_DIR  = "../results"
METHOD_NAME  = "fourier_umap_hdbscan_2"

# Grille d'hyperparamètres
N_FOURIER_COEFFS_LIST       = [8, 16, 24]

UMAP_N_COMPONENTS_LIST      = [2, 3]       
UMAP_N_NEIGHBORS_LIST       = [15]
UMAP_MIN_DIST_LIST          = [0.0, 0.1]
UMAP_METRIC_LIST            = ["euclidean", "manhattan", "cosine"] 
HDBSCAN_MIN_CLUSTER_SIZE_LIST = [20, 30, 40]
HDBSCAN_MIN_SAMPLES_LIST      = [5, 10, 15, 20]
HDBSCAN_METRIC_LIST           = ["euclidean", "manhattan"]

RANDOM_SEEDS = [42]  

SAMPLING_RATE    = 51.2
COMPUTE_EMBEDDING = True
SAVE_FIGURES      = True
SUBSAMPLE_RATIO   = 1.0

In [ ]:
Xn_train = z_norm_per_series(X_train)
Xn_test  = z_norm_per_series(X_test)

freqs, amp = compute_fft_spectrum(
    X=Xn_train,
    sampling_rate=SAMPLING_RATE,
    center=True
)

X_time = compute_time_features(Xn_train)

ytrue_full = y_train.copy()

#Subsample for faster experiments
if SUBSAMPLE_RATIO < 1.0:
    rng = np.random.default_rng(0)
    idx = rng.choice(len(ytrue_full), size=int(len(ytrue_full) * SUBSAMPLE_RATIO), replace=False)
    X_time     = X_time[idx]
    amp        = amp[idx]
    ytrue_full = ytrue_full[idx]

In [ ]:
def run_one(n_fourier_coeffs,
            umap_n_components, umap_n_neighbors, umap_min_dist, umap_metric,
            hdb_min_cluster_size, hdb_min_samples, hdb_metric,
            seed):

    t0 = time.time()

    # ---- Features freq ----
    X_freq = extract_fourier_features(amp, n_coeffs=n_fourier_coeffs)

    # ---- Concat time + freq ----
    X_feat = concatenate_features(X_time, X_freq)

    # ---- Scaling ----
    X_feat_scaled = scale_features(X_feat)

    # ---- UMAP ----
    X_umap = apply_umap(
        X=X_feat_scaled,
        n_components=umap_n_components,
        n_neighbors=umap_n_neighbors,
        min_dist=umap_min_dist,
        metric=umap_metric,
        random_state=seed
    )

    # ---- HDBSCAN ----
    y_pred, clusterer = run_hdbscan(
        X=X_umap,
        min_cluster_size=hdb_min_cluster_size,
        min_samples=hdb_min_samples,
        metric=hdb_metric
    )

    # ---- Gestion du bruit ----
    mask = (y_pred != -1)
    n_noise = np.sum(~mask)

    params = {
        "n_fourier_coeffs": n_fourier_coeffs,
        "umap_n_components": umap_n_components,
        "umap_n_neighbors": umap_n_neighbors,
        "umap_min_dist": umap_min_dist,
        "umap_metric": umap_metric,
        "hdb_min_cluster_size": hdb_min_cluster_size,
        "hdb_min_samples": hdb_min_samples,
        "hdb_metric": hdb_metric,
        "random_state": seed,
        "sampling_rate": SAMPLING_RATE,
        "metric": "fourier_umap_hdbscan",
    }

    #trop peu de points non bruit ou 1 seul cluster
    if mask.sum() < 2 or len(np.unique(y_pred[mask])) < 2:
        scores = {
            "ARI": np.nan,
            "NMI": np.nan,
            "Silhouette_UMAP": np.nan,
        }
        elapsed = time.time() - t0

        # pas de sauvegarde utile ici
        return {
            "run_dir": None,
            "run_id": None,
            "params": params,
            "scores": scores,
            "time_sec": elapsed,
            "n_noise": n_noise,
            "n_clusters_found": 0,
        }

    #au moins 2 clusters non bruit
    ytrue = ytrue_full[mask]
    y_pred_masked = y_pred[mask]

    # Scores externes
    scores = external_scores(ytrue, y_pred_masked)

    # Silhouette dans l'espace UMAP
    sil = silhouette_by_metric(X_umap[mask], y_pred_masked, metric=hdb_metric)
    scores["Silhouette_UMAP"] = float(sil)

    # Hungarian + confusion
    y_pred_mapped, C, mapping = confusion_with_hungarian(ytrue, y_pred_masked)
    mapping_clean = {int(k): int(v) for k, v in mapping.items()}

    # t-SNE (optionnel)
    X2 = None
    D = None
    if COMPUTE_EMBEDDING:
        D = distance_matrix(X_umap[mask], metric=hdb_metric)
        X2 = embed_tsne_from_dist(D, perplexity=30, random_state=seed)

    # "Centres" HDBSCAN (barycentres )
    unique_clusters = np.unique(y_pred_masked)
    centers_umap = []
    for c in unique_clusters:
        centers_umap.append(X_umap[mask][y_pred_masked == c].mean(axis=0))
    centers_umap = np.vstack(centers_umap) if len(centers_umap) > 0 else None

    # Sauvegarde
    root, arr_dir, fig_dir, run_id = save_clustering_results(
        method_name=METHOD_NAME,
        y_pred=y_pred_masked,
        y_true=ytrue,
        scores=scores,
        centers=centers_umap,  
        mapping=mapping_clean,
        embedding_2d=X2,
        dist_matrix=D,
        confusion=C,
        labels_true=np.sort(np.unique(ytrue)),
        labels_pred=np.sort(np.unique(y_pred_masked)),
        params=params,
        output_dir=RESULTS_DIR,
        run_id=None,
    )
    if SAVE_FIGURES and X2 is not None:
        import matplotlib.pyplot as plt

        # 1) t-SNE scatter
        fig = scatter_2d(
            X2,
            y_pred_masked,
            title=(f"HDBSCAN — t-SNE (FFT={n_fourier_coeffs}, "
                f"UMAP_dim={umap_n_components}, "
                f"min_cl_size={hdb_min_cluster_size})"),
            y_true=ytrue,
            show=False
        )
        fig.savefig(f"{fig_dir}/tsne.png", dpi=150)
        plt.close(fig)

        # 2) Confusion matrix
        true_labels = np.sort(np.unique(ytrue))
        pred_labels = np.sort(np.unique(y_pred_masked))

        fig = plot_confusion_matrix(
            C,
            labels_true=true_labels,
            labels_pred=pred_labels,
            label_to_activity={1:"Walk",2:"Stairs Up",3:"Stairs Down",
                            4:"Sit",5:"Stand",6:"Lay"},
            normalize=True,
            title="HDBSCAN Confusion Matrix (Hungarian Aligned)",
            show=False
        )
        fig.savefig(f"{fig_dir}/confusion.png", dpi=150)
        plt.close(fig)


    elapsed = time.time() - t0

    return {
        "run_dir": root,
        "run_id": run_id,
        "params": params,
        "scores": scores,
        "time_sec": elapsed,
        "n_noise": n_noise,
        "n_clusters_found": int(len(unique_clusters)),
    }


In [ ]:
results = []

for (n_fourier_coeffs,
     umap_n_components, umap_n_neighbors, umap_min_dist, umap_metric,
     hdb_min_cluster_size, hdb_min_samples, hdb_metric,
     seed) in product(
        N_FOURIER_COEFFS_LIST,
        UMAP_N_COMPONENTS_LIST,
        UMAP_N_NEIGHBORS_LIST,
        UMAP_MIN_DIST_LIST,
        UMAP_METRIC_LIST,
        HDBSCAN_MIN_CLUSTER_SIZE_LIST,
        HDBSCAN_MIN_SAMPLES_LIST,
        HDBSCAN_METRIC_LIST,
        RANDOM_SEEDS
    ):

    print(
        f"--> n_fft={n_fourier_coeffs} | "
        f"UMAP(d={umap_n_components}, nn={umap_n_neighbors}, min_dist={umap_min_dist}, metric={umap_metric}) | "
        f"HDBSCAN(min_cl_size={hdb_min_cluster_size}, min_samples={hdb_min_samples}, metric={hdb_metric}) | "
        f"seed={seed}"
    )

    out = run_one(
        n_fourier_coeffs=n_fourier_coeffs,
        umap_n_components=umap_n_components,
        umap_n_neighbors=umap_n_neighbors,
        umap_min_dist=umap_min_dist,
        umap_metric=umap_metric,
        hdb_min_cluster_size=hdb_min_cluster_size,
        hdb_min_samples=hdb_min_samples,
        hdb_metric=hdb_metric,
        seed=seed
    )
    results.append(out)

    ARI = out["scores"].get("ARI", np.nan)
    NMI = out["scores"].get("NMI", np.nan)
    Sil = out["scores"].get("Silhouette_UMAP", np.nan)
    print(
        f"ARI={ARI:.3f}  "
        f"NMI={NMI:.3f}  "
        f"Sil={Sil:.3f}  "
        f"noise={out['n_noise']}  "
        f"n_clusters={out['n_clusters_found']}  "
        f"time={out['time_sec']:.1f}s"
    )

--> n_fft=8 | UMAP(d=2, nn=15, min_dist=0.0, metric=euclidean) | HDBSCAN(min_cl_size=20, min_samples=5, metric=euclidean) | seed=42


c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ Saved run to: ../results\fourier_umap_hdbscan_2\2025-11-15_11-56-57_seed42
ARI=0.656  NMI=0.831  Sil=0.795  noise=0  n_clusters=4  time=1.0s
--> n_fft=8 | UMAP(d=2, nn=15, min_dist=0.0, metric=euclidean) | HDBSCAN(min_cl_size=20, min_samples=5, metric=manhattan) | seed=42


c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ Saved run to: ../results\fourier_umap_hdbscan_2\2025-11-15_11-56-58_seed42
ARI=0.656  NMI=0.831  Sil=0.810  noise=0  n_clusters=4  time=1.1s
--> n_fft=8 | UMAP(d=2, nn=15, min_dist=0.0, metric=euclidean) | HDBSCAN(min_cl_size=20, min_samples=10, metric=euclidean) | seed=42


...

✅ Saved run to: ../results\fourier_umap_hdbscan_2\2025-11-15_12-15-06_seed42
ARI=0.511  NMI=0.703  Sil=0.728  noise=0  n_clusters=3  time=1.2s
--> n_fft=24 | UMAP(d=3, nn=15, min_dist=0.1, metric=cosine) | HDBSCAN(min_cl_size=40, min_samples=20, metric=euclidean) | seed=42


c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ Saved run to: ../results\fourier_umap_hdbscan_2\2025-11-15_12-15-07_seed42
ARI=0.511  NMI=0.703  Sil=0.700  noise=0  n_clusters=3  time=1.1s
--> n_fft=24 | UMAP(d=3, nn=15, min_dist=0.1, metric=cosine) | HDBSCAN(min_cl_size=40, min_samples=20, metric=manhattan) | seed=42


c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\projetsUniversite\Master_2\clustering\human_activity_clustering\venv311\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


✅ Saved run to: ../results\fourier_umap_hdbscan_2\2025-11-15_12-15-09_seed42
ARI=0.347  NMI=0.561  Sil=0.741  noise=0  n_clusters=2  time=1.1s


In [11]:
try:
    rows = []
    for r in results:
        row = {
            **r["params"],
            **r["scores"],
            "time_sec": r["time_sec"],
            "n_noise": r["n_noise"],
            "n_clusters_found": r["n_clusters_found"],
            "run_dir": r["run_dir"]
        }
        rows.append(row)
    df = pd.DataFrame(rows).sort_values(
        ["ARI", "NMI", "Silhouette_UMAP"],
        ascending=False
    )
    display(df)
except Exception:
    print("Install pandas to get a sortable summary table.")


,n_fourier_coeffs,umap_n_components,umap_n_neighbors,umap_min_dist,umap_metric,hdb_min_cluster_size,hdb_min_samples,hdb_metric,random_state,sampling_rate,metric,ARI,NMI,Purity,Silhouette_UMAP,time_sec,n_noise,n_clusters_found,run_dir
600,24,2,15,0.0,manhattan,20,5,euclidean,42,51.2,fourier_umap_hdbscan,0.758439,0.787279,0.795367,0.644695,1.254978,18,5,../results\fourier_umap_hdbscan_2\2025-11-15_1...
192,8,3,15,0.0,cosine,20,5,euclidean,42,51.2,fourier_umap_hdbscan,0.663822,0.843258,0.718412,0.853967,1.471025,0,4,../results\fourier_umap_hdbscan_2\2025-11-15_1...
194,8,3,15,0.0,cosine,20,10,euclidean,42,51.2,fourier_umap_hdbscan,0.663822,0.843258,0.718412,0.853967,1.263933,0,4,../results\fourier_umap_hdbscan_2\2025-11-15_1...
196,8,3,15,0.0,cosine,20,15,euclidean,42,51.2,fourier_umap_hdbscan,0.663822,0.843258,0.718412,0.853967,1.434029,0,4,../results\fourier_umap_hdbscan_2\2025-11-15_1...
198,8,3,15,0.0,cosine,20,20,euclidean,42,51.2,fourier_umap_hdbscan,0.663822,0.843258,0.718412,0.853967,1.232159,0,4,../results\fourier_umap_hdbscan_2\2025-11-15_1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,24,3,15,0.1,euclidean,40,15,manhattan,42,51.2,fourier_umap_hdbscan,0.321344,0.516165,0.422383,0.644941,1.144340,0,2,../results\fourier_umap_hdbscan_2\2025-11-15_1...
815,24,3,15,0.1,euclidean,40,20,manhattan,42,51.2,fourier_umap_hdbscan,0.321344,0.516165,0.422383,0.644941,1.253913,0,2,../results\fourier_umap_hdbscan_2\2025-11-15_1...
649,24,2,15,0.1,euclidean,20,5,manhattan,42,51.2,fourier_umap_hdbscan,0.304598,0.605202,0.467391,0.563962,1.189924,1,4,../results\fourier_umap_hdbscan_2\2025-11-15_1...
576,24,2,15,0.0,euclidean,20,5,euclidean,42,51.2,fourier_umap_hdbscan,0.299551,0.602682,0.461818,0.676532,1.210887,2,4,../results\fourier_umap_hdbscan_2\2025-11-15_1...
